In [233]:
import requests
import json
import boto3

In [234]:

ACCESS_KEY = ''
SECRET_KEY = ''
API_KEY = ''


In [235]:
# S3
REGION = 'ca-central-1'
BUCKET = 'quetzal-immense'

# api gateway
url = 'https://lud5uqi5j5.execute-api.ca-central-1.amazonaws.com/dev'
stateMachineArn = 'arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense'

In [236]:
headers = {
        'Accept': '*/*',
        'Content-Type':'application/json',
        'x-api-key' : API_KEY,
}

# S3 structure

scenario/inputs/pt/links.geojson <br>
scenario/inputs/pt/nodes.geojson <br>
scenario/inputs/road/road_links.geojson <br>
scenario/inputs/road/road_nodes.geojson <br>
scenario/inputs/car_skim.csv <br>
scenario/outputs/<br>
<br>
you may see other files (such as params.json) those are either used in the web interface are create by the model.

In [282]:
periods = ['am', 'pm']

In [283]:
session = boto3.Session(
    region_name=REGION,
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY)

s3 = session.client('s3')

In [288]:
inputs_list=[]
for period in periods:
    prefix = period + '/inputs/'
    response = s3.list_objects_v2(Bucket=BUCKET, Prefix=prefix)
    for obj in response['Contents']:
        inputs_list.append(obj["Key"])

In [289]:
inputs_list

['am/inputs/car_skim.csv',
 'am/inputs/params.json',
 'am/inputs/pt/links.geojson',
 'am/inputs/pt/nodes.geojson',
 'am/inputs/road/road_links.geojson',
 'am/inputs/road/road_nodes.geojson',
 'pm/inputs/car_skim.csv',
 'pm/inputs/params.json',
 'pm/inputs/pt/links.geojson',
 'pm/inputs/pt/nodes.geojson',
 'pm/inputs/road/road_links.geojson',
 'pm/inputs/road/road_nodes.geojson']

<b>NOTE:</b> Constant inputs may be store into the model docker directly.<br>
zones, reference volume and refereen skims are store into the docker and not on S3

At this moment, Only one period can be run per scenario. <br>
the solution is to run a scenario per period <br>
and have another scenario (running the demand model) that will used data from those period and return the outputs <br>
<br>
In this example, this mean that we need to add our inputs under am/ and pm/ and run a third model (named example here). This model will run both am and pm in parallel and run the demand model. the results will be under example/outputs/
<br>
<br>
even tho the last scenario (example) can have any name, the period one cannot at this moment. this mean that you can only run one simulation at the time as they all use the same s3 folder.

# Run

In [263]:
scenario = 'example/'

In [264]:
inputs = { 
        'scenario_path_S3': scenario,
        'choice': 'orchestrator',
        'metadata' : {"user_email": "test@test.com"}
}
inputs = json.dumps(inputs)


In [265]:
data = { "stateMachineArn": stateMachineArn, 'input':inputs }
data = json.dumps(data)
data

'{"stateMachineArn": "arn:aws:states:ca-central-1:142023388927:stateMachine:quetzal-immense", "input": "{\\"scenario_path_S3\\": \\"example/\\", \\"choice\\": \\"orchestrator\\", \\"metadata\\": {\\"user_email\\": \\"test@test.com\\"}}"}'

POST start quetzal model execution

In [266]:
resp =  requests.post(url,data=data,headers=headers)
resp.ok

True

execution arn is returned in the post response and used to get status or end the simulation

In [267]:
executionArn = resp.json()['executionArn']
executionArn

'arn:aws:states:ca-central-1:142023388927:execution:quetzal-immense:0233dada-3370-40d9-8c9c-41a4bd6e5e9b'

# Polling Simulation Status

RUNNING | SUCCEEDED | FAILED | TIMED_OUT | ABORTED 

In [249]:

data = { "executionArn": executionArn }
data=json.dumps(data, ensure_ascii=False).encode('utf-8')
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

status =  'RUNNING'
while status == 'RUNNING':
    resp =  requests.post(url+'/describe',data=data,headers=headers)
    status = resp.json()['status']
    print(status)
    time.sleep(10)

RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
RUNNING
SUCCEEDED


You could get the step function definition and poll its current Step if needed. <br>
However, it is a bit difficult to decrypt and not really relevent for you in my opinion.


# get output

In [272]:
prefix = scenario + 'outputs'
response = s3.list_objects_v2(Bucket=BUCKET,Prefix=scenario+'outputs')
for obj in response['Contents']:
    print(obj["Key"])

example/outputs/volume_am.csv
example/outputs/volume_pm.csv


# Stop simulation

In [268]:
data = { "executionArn": executionArn }
data = json.dumps(data)
resp =  requests.post(url+'/abort',data=data,headers=headers)
resp.ok


True

In [269]:
resp.json()

{'stopDate': 1707750391.588}

In [270]:
resp =  requests.post(url+'/describe',data=data,headers=headers)
resp.ok

True

In [271]:
resp.json()['status'] # will return ABORTED. if not. the simulation was done are had an error.

'ABORTED'